In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
positive_dataset = pd.read_csv('/content/drive/MyDrive/combined_data1.csv')
positive_dataset.head()

,Unnamed: 0,Unnamed: 0.1,uniref100,sequence,disorder,len
0,0,0,UniRef100_P18075,MHVRSLRAAAPHSFVALWAPLFLLRSALADFSLDNEVHSSFIHRRL...,0.146333,431
1,1,1,UniRef100_P12644,MIPGNRMLMVVLLCQVLLGGASHASLIPETGKKKVAEIQGHAGGRR...,0.244167,408
2,2,2,UniRef100_Q15848,MLLLGAVLLLLALPGHDQETTTQGPGVLLPLPKGACTGWMAGIPGH...,0.315333,244
3,3,3,UniRef100_P05305,MDYLLMIFSLLFVACQGAPETAVLGAELSAVGENGGEKPTPSPPWR...,0.382083,212
4,4,4,UniRef100_P17787,MARRCGPVALLLGFGLLRLCSGVWGTDTEERLVEHLLDPSRYNKLI...,0.106167,502


In [4]:
positive_dataset=positive_dataset[positive_dataset.len<=1000]

In [5]:
f = open('/content/drive/MyDrive/protein_Fatemeh_vec.txt')
lines = f.readlines()
vecs=[]
for line in lines:
    vec =[]
    elms = line.split()
    vec=np.array(elms[1:]).astype(float)
    vecs.append(vec)
positive_dataset['sequence']=vecs
# dataset['disorder'] =dataset['disorder'].apply(lambda x: 1 if x >= .3 else 0)      
positive_dataset

,Unnamed: 0,Unnamed: 0.1,uniref100,sequence,disorder,len
0,0,0,UniRef100_P18075,"[-0.015283266, 0.03260137, 0.05206222, -0.0242...",0.146333,431
1,1,1,UniRef100_P12644,"[-0.045884732, 0.020690238, 0.07738209, 0.0120...",0.244167,408
2,2,2,UniRef100_Q15848,"[-0.02228709, -0.011490774, 0.024070028, -0.01...",0.315333,244
3,3,3,UniRef100_P05305,"[-0.0070567443, -0.027871102, 0.044447333, -0....",0.382083,212
4,4,4,UniRef100_P17787,"[-0.017075546, 0.0098427115, 0.0438065, -0.032...",0.106167,502
...,...,...,...,...,...,...
682,682,682,UniRef100_P09327,"[0.05533529, 0.0032542942, -0.03586592, 0.0066...",0.307134,827
683,683,683,UniRef100_Q13740,"[0.024731068, 0.069300026, -0.030226637, -0.02...",0.579760,583
684,684,684,UniRef100_Q9UKL0,"[0.04175786, 0.01022134, -0.0057291063, 0.0175...",0.725773,485
686,686,686,UniRef100_P18564,"[0.013786916, -0.080442995, -0.032576166, 0.01...",0.376904,788


In [6]:
negative_dataset = pd.read_csv('/content/drive/MyDrive/neg_data.csv')
negative_dataset.head()

,Unnamed: 0,index,seq,len
0,0,0,SMNPPPPETSNPNKPKRQTNQLQYLLRVVLKTLWKHQFAWPFQQPV...,127
1,1,1,SMNPPPPETSNPNKPKRQTNQLQYLLRVVLKTLWKHQFAWPFQQPV...,127
2,2,2,SMNPPPPETSNPNKPKRQTNQLQYLLRVVLKTLWKHQFAWPFQQPV...,127
3,3,3,GSAPAKEPSVNTALVPQLSTISRALTPSPVMVLENIEPEIVYAGYD...,275
4,4,4,HMTVFRQENVDDYYDTGEELGSGQFAVVKKCREKSTGLQYAAKFIK...,278


In [7]:
negative_dataset=negative_dataset[negative_dataset.len<=1000]

In [9]:
f = open('/content/drive/MyDrive/Fatemeh_neg_data.txt')
lines = f.readlines()

In [12]:
lines[-1]

'4084\t0.006581166\t-0.021784823\t0.09137244\t-0.0011015192\t0.05403619\t-0.038999382\t0.029594345\t-0.016030964\t-0.001335132\t0.046185467\t-0.055104043\t0.061569054\t-0.013443274\t0.0029690359\t-0.036028385\t0.031134965\t0.04871293\t-0.08846908\t-0.11874799\t0.044609718\t-0.050741278\t0.0022330428\t-0.01147979\t0.031754404\t-0.02205421\t-0.029170357\t-0.021603767\t-0.21538757\t0.011778778\t0.091709495\t-0.020833196\t0.0058783833\t0.00432938\t0.10557047\t0.09850161\t-0.016580246\t0.011158898\t0.03219912\t-0.033787053\t0.06381909\t-0.11211123\t0.063279994\t-0.040388383\t-0.14279652\t-0.14757992\t-0.12538047\t0.08151175\t-0.017794799\t0.0067921323\t-0.038313482\t-0.027258944\t0.0698573\t0.022476701\t0.1202303\t-0.030615985\t0.06162813\t-0.028601855\t-0.013712705\t-0.015542854\t0.021866107\t0.023545122\t-0.012949279\t0.030843724\t0.056617722\t0.01888411\t0.05062549\t-0.10950451\t0.01580622\t-0.032896977\t-0.027048519\t0.06862832\t0.08545377\t-0.019056918\t-0.08205701\t-0.012588497\t-0.03

In [8]:
f = open('/content/drive/MyDrive/Fatemeh_neg_data.txt')
lines = f.readlines()
vecs=[]
for line in lines:
    vec =[]
    elms = line.split()
    vec=np.array(elms[1:]).astype(float)
    vecs.append(vec)
negative_dataset['sequence']=vecs
# dataset['disorder'] =dataset['disorder'].apply(lambda x: 1 if x >= .3 else 0)      
negative_dataset

ValueError: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['sequence'], dataset['disorder'], test_size=0.2, random_state=42)

In [ ]:
max_features = 4096

In [ ]:
inputs = keras.Input(shape=(None,), dtype="float64")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)

x = layers.Bidirectional(layers.LSTM(32))(x)
# x = layers.Bidirectional(layers.LSTM(32))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 128)         524288    
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 763,201
Trainable params: 763,201
Non-trainable params: 0
_____________________________________________________

In [ ]:
# print(len(X_train), "Training sequences")
# print(len(X_test), "Validation sequences")
features = tf.ragged.stack(list(X_train))
target = tf.convert_to_tensor(np.asarray((y_train.values).astype('float32').reshape((-1,1))))
train_dataset = tf.data.Dataset.from_tensor_slices((features, target))

In [ ]:
features = tf.ragged.stack(list(X_test))
target = tf.convert_to_tensor(np.asarray((y_test.values).astype('float32').reshape((-1,1))))
test_dataset = tf.data.Dataset.from_tensor_slices((features, target))

In [ ]:
train_dataset

<TensorSliceDataset shapes: ((None,), (1,)), types: (tf.float64, tf.float32)>

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(train_dataset, batch_size=32, epochs=2, validation_data=test_dataset)